In [4]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

size = 96
font_size = 96
font = ImageFont.truetype("arial.ttf", font_size)
output_dir = "digit_headers"
os.makedirs(output_dir, exist_ok=True)

def bmp_to_custom_header(img, digit):
    # ensure 1-bit image
    img = img.convert('1')
    data = np.array(img)         # shape (height, width), 0=black, 255=white
    height, width = data.shape

    var_name     = f"digit_{digit}"
    guard        = var_name.upper() + "_H_"
    
    # bytes per row (8 pixels per byte)
    row_bytes = (width + 7) // 8

    bytes_list = []
    for y in range(height):
        for xb in range(0, width, 8):
            byte = 0
            for bit in range(8):
                x = xb + bit
                if x < width and data[y, x] == 0:
                    # MSB = leftmost pixel
                    byte |= 1 << (7 - bit)
            bytes_list.append(byte)

    # format into hex strings, 12 per line
    hex_strs = [f"0x{b:02X}" for b in bytes_list]
    lines = [", ".join(hex_strs[i:i+12]) for i in range(0, len(hex_strs), 12)]

    header = []
    header.append(f"#ifndef _{guard}")
    header.append(f"#define _{guard}\n")
    header.append(f"#define {var_name}_width  {width}")
    header.append(f"#define {var_name}_height {height}\n")
    header.append(f"static const uint8_t PROGMEM {var_name}_data[] = {{")
    for line in lines:
        header.append("  " + line + ",")
    header[-1] = header[-1].rstrip(',')  # remove trailing comma on last line
    header.append("};\n")
    header.append(f"#endif // _{guard}")

    return "\n".join(header)

# Generate all digits
for i in range(10):
    img = Image.new('1', (size, size), 1)
    draw = ImageDraw.Draw(img)
    text = str(i)
    bbox = draw.textbbox((0, 0), text, font=font)
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    x = (size - w) // 2 - bbox[0]
    y = (size - h) // 2 - bbox[1]
    draw.text((x, y), text, font=font, fill=0)

    header_content = bmp_to_custom_header(img, i)
    with open(os.path.join(output_dir, f"digit_{i}.h"), "w") as f:
        f.write(header_content)
    print(f"Written digit_{i}.h")


Written digit_0.h
Written digit_1.h
Written digit_2.h
Written digit_3.h
Written digit_4.h
Written digit_5.h
Written digit_6.h
Written digit_7.h
Written digit_8.h
Written digit_9.h
